In [ ]:
!pip install transformers
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install accelerate
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install transformers==4.30
!pip install datasets

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoTokenizer, LlamaForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = LlamaForCausalLM.from_pretrained("kaist-ai/Prometheus-7b-v1.0", cache_dir="/content/drive/MyDrive/models", load_in_4bit=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#!git clone https://github.com/agokrani/eval-note-generation.git

Cloning into 'eval-note-generation'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 1), reused 9 (delta 1), pack-reused 0
Receiving objects: 100% (12/12), 198.02 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
import os
import re
import json
from datasets import load_dataset
from string import Template

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/git/eval-note-generation/outputs/train.gpt-4-1106-preview.pred.fullnote.json'
TEMP_PATH = '/content/drive/MyDrive/git/eval-note-generation/outputs/temp/prompt-1'
dataset = load_dataset('json', data_files=DATASET_PATH)
dataset = dataset['train']

In [ ]:
PROMPT = Template(
"""
###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
4. Please do not generate any other opening, closing, and explanations.

###The instruction to evaluate:
summarize the conversation to generate a clinical note with four sections: HISTORY OF PRESENT ILLNESS, PHYSICAL EXAM, RESULTS, ASSESSMENT AND PLAN.
The conversation is:
$conversation

###Response to evaluate:
$summary

###Reference Answer (Score 5):
$reference

###Score Rubric:
[Is the model able to accurately and effectively summarize a medical conversation into a clinical note with four sections: HISTORY OF PRESENT ILLNESS, PHYSICAL EXAM, RESULTS, ASSESSMENT AND PLAN?]

Score 1: The summary utterly fails to reflect the conversation. It is incoherent, irrelevant, excessively verbose, or filled with hallucinations. There is a blatant disregard for standard clinical terminology, and critical information is omitted.

Score 2: The summary sporadically reflects elements of the conversation, but it frequently includes irrelevant or incoherent content. There is a noticeable lack of standard clinical terminology, verbosity is apparent, hallucinations are present, and critical information is often omitted.

Score 3: The summary generally captures the conversation accurately but occasionally includes irrelevant or incoherent content. It mostly uses standard clinical terms but can be verbose at times. Minor hallucinations may occur, and there might be instances of critical information being overlooked.

Score 4: The summary often accurately reflects the conversation, maintaining coherence and relevance throughout. There are minor cases of verbosity or use of non-standard clinical terms. The summary may have slight omissions or infrequent minor hallucinations.

Score 5: The summary flawlessly encapsulates the conversation, demonstrating complete coherence, relevance, and succinctness. It consistently employs standard clinical terminology, contains no hallucinations, and does not omit any critical information.

###Feedback:
"""
)

In [ ]:
def parse_feedback(input_string):
    """
    Parses the string to extract the content that comes after '###Feedback:'.

    Args:
    input_string (str): The input string containing the '###Feedback:' marker.

    Returns:
    str: The substring that comes after '###Feedback:'.
    """
    # Define the marker
    marker = "###Feedback:"

    # Find the index of the marker
    index = input_string.find(marker)

    # Check if the marker is found
    if index != -1:
        # Extract and return the substring that comes after the marker
        return input_string[index + len(marker):].strip()
    else:
        # Return an empty string if the marker is not found
        return ""


In [ ]:
for index, example in enumerate(dataset):
  temp_file = os.path.join(TEMP_PATH, f"{example['file']}.json")
  print("processing file: ", temp_file)
  if not os.path.exists(temp_file):
    conversation = example['src'].replace('[doctor]', 'doctor:').replace('[patient]', 'patient:')
    conversation = re.sub(r'\s+([,.?])', r'\1', conversation)
    reference = example['tgt']
    summary = example['pred']

    #PROMPT_WITH_CON = PROMPT.substitute(conversation=conversation, reference=reference, summary=summary)
    PROMPT_WITHOUT_CON = PROMPT.substitute(conversation='', reference=reference, summary=summary)
    #input_text = [PROMPT_WITH_CON, PROMPT_WITHOUT_CON]
    input_text = PROMPT_WITHOUT_CON
    input_ids = tokenizer(input_text, padding=True, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, temperature=1.0, top_p=0.4, max_new_tokens=512, repetition_penalty=1.03)
    outputs = parse_feedback(tokenizer.decode(outputs[0]))

    with open(temp_file, "w") as f:
      json.dump([outputs], f, indent=4)



processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N001-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N002-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N003-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N004-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N005-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N006-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N007-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N008-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N009-virtassist.json
processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/D2N

In [ ]:
!nvidia-smi
!nvidia-top

Sat Dec  2 15:29:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |  14539MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#for index, example in enumerate(dataset):
for index in range(20, len(dataset)):
  example = dataset[index]
  temp_file = os.path.join(TEMP_PATH, f"{example['file']}.json")
  print("processing file: ", temp_file)
  if os.path.exists(temp_file):
    conversation = example['src'].replace('[doctor]', 'doctor:').replace('[patient]', 'patient:')
    conversation = re.sub(r'\s+([,.?])', r'\1', conversation)
    reference = example['tgt']
    summary = example['pred']

    PROMPT_WITH_CON = PROMPT.substitute(conversation=conversation, reference=reference, summary=summary)
    input_text = PROMPT_WITH_CON
    input_ids = tokenizer(input_text, padding=True, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, temperature=1.0, top_p=0.4, max_new_tokens=512, repetition_penalty=1.03)
    outputs = parse_feedback(tokenizer.decode(outputs[0]))

    with open(temp_file, "r") as file:
      data = json.load(file)
    if isinstance(data, list):
      # Append the new item to the list
      data.append(outputs)
    with open(temp_file, "w") as file:
      json.dump(data, file, indent=4)



In [ ]:
example = dataset[0]
temp_file = os.path.join(TEMP_PATH, f"{example['file']}.json")
print("processing file: ", temp_file)
if os.path.exists(temp_file):
  conversation = example['src'].replace('[doctor]', 'doctor:').replace('[patient]', 'patient:')
  conversation = re.sub(r'\s+([,.?])', r'\1', conversation)
  reference = example['tgt']
  summary = example['pred']

  PROMPT_WITH_CON = PROMPT.substitute(conversation=conversation, reference=reference, summary=summary)
  input_text = PROMPT_WITH_CON
  input_ids = tokenizer(input_text, padding=True, return_tensors="pt").input_ids
  outputs = model.generate(input_ids, temperature=1.0, top_p=0.4, max_new_tokens=512, repetition_penalty=1.03)
  outputs = parse_feedback(tokenizer.decode(outputs[0]))

  with open(temp_file, "r") as file:
    data = json.load(file)
  if isinstance(data, list):
    # Append the new item to the list
    data.append(outputs)
  print(data)

processing file:  /content/drive/MyDrive/git/eval-note-generation/outputs/temp/prompt-1/D2N001-virtassist.json


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1452: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


["The response accurately reflects the conversation, maintaining coherence and relevance throughout. It employs standard clinical terminology and does not contain any hallucinations. However, there are instances of verbosity, such as the inclusion of unnecessary details about the patient's travel history and hiking activities. The summary could have been more concise while still providing the necessary information. Additionally, the response could have better emphasized the patient's inconsistent medication adherence and the need for improved adherence. Overall, the response demonstrates a good understanding of the conversation but could have been more concise and focused on the essential points. So the overall score is 4. [RESULT] 4</s>", 'The summary does not accurately reflect the conversation. It includes irrelevant and incoherent content, and there is a noticeable lack of standard clinical terminology. The verbosity is apparent, and there are instances of critical information bein

In [ ]:
input_text = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
4. Please do not generate any other opening, closing, and explanations.

###The instruction to evaluate:
summarize the conversation to generate a clinical note with four sections: HISTORY OF PRESENT ILLNESS, PHYSICAL EXAM, RESULTS, ASSESSMENT AND PLAN.
The conversation is:
doctor: hi, martha. how are you?
patient: i'm doing okay. how are you?
doctor: i'm doing okay. so, i know the nurse told you about dax. i'd like to tell dax a little bit about you, okay?
patient: okay.
doctor: martha is a 50-year-old female with a past medical history significant for congestive heart failure, depression and hypertension who presents for her annual exam. so, martha, it's been a year since i've seen you. how are you doing?
patient: i'm doing well. i've been traveling a lot recently since things have, have gotten a bit lighter. and i got my, my vaccine, so i feel safer about traveling. i've been doing a lot of hiking. uh, went to washington last weekend to hike in northern cascades, like around the mount baker area.
doctor: nice. that's great. i'm glad to hear that you're staying active, you know. i, i just love this weather. i'm so happy the summer is over. i'm definitely more of a fall person.
patient: yes, fall foliage is the best.
doctor: yeah. um, so tell me, how are you doing with the congestive heart failure? how are you doing watching your diet? i know we've talked about watching a low sodium diet. are you doing okay with that?
patient: i've been doing well with that. i resisted, as much, as i could, from the tater tots, you know, the soft pretzels, the salty foods that i, i love to eat. and i've been doing a really good job.
doctor: okay, all right. well, i'm glad to hear that. and you're taking your medication?
patient: yes.
doctor: okay, good. and any symptoms like chest pains, shortness of breath, any swelling in your legs?
patient: no, not that i've noticed.
doctor: okay, all right. and then in terms of your depression, i know that we tried to stay off of medication in the past because you're on medications for your other problems. how are you doing? and i know that you enrolled into therapy. is that helping? or-
patient: yeah, it's been helping a lot. i've been going every week, um, for the past year since my last annual exam. and that's been really helpful for me.
doctor: okay. so, no, no issues, no feelings of wanting to harm yourself or hurt others?
patient: no, nothing like that.
doctor: okay, all right. and then in terms of your high blood pressure, i know that you and i have kind of battled in the past with you remembering to take some of your blood pressure medications. how are you doing with that?
patient: i'm still forgetting to take my blood pressure medication. and i've noticed when work gets more stressful, my blood pressure goes up.
doctor: okay. and, and so how has work going for you?
patient: it's been okay. it's been a lot of long hours, late nights. a lot of, um, you know, fiscal year end data that i've been having to pull. so, a lot of responsibility, which is good. but with the responsibility comes the stress.
doctor: yeah, okay, all right. i understand. um, all right. well, i know that you did a review of system sheet when you checked in with the nurse. i know that you were endorsing some nasal congestion from some of the fall pollen and allergies. any other symptoms, nausea or vomiting, abdominal pain, anything like that?
patient: no, nothing like that.
doctor: no, okay, all right. well, i'm gon na go ahead and do a quick physical exam, okay?
patient: okay.
doctor: hey, dragon, show me the blood pressure. so, yeah, looking at your blood pressure today here in the office, it is a little elevated. you know, it could just, you could just be nervous. uh, let's look at some of the past readings. hey, dragon, show me the blood pressure readings. hey, dragon, show me the blood pressure readings. here we go. uh, so they are running on the higher side. um, y- you know, i, i do think that, you know, i'd like to see you take your medication a little bit more, so that we can get that under control a little bit better, okay?
patient: okay.
doctor: so, i'm just gon na check out your heart and your lungs. and you know, let you know what i find, okay?
patient: okay.
doctor: okay. so, on your physical examination, you know, everything looks good. on your heart exam, i do appreciate a three out of six systolic ejection murmur, which i've heard in the past, okay? and on your lower extremities, i do appreciate one plus pitting edema, so you do have a little bit of fluid in your legs, okay?
patient: okay.
doctor: let's go ahead, i wan na look at some of your results, okay? hey, dragon, show me the echocardiogram. so, this is the result of the echocardiogram that we did last year. it showed that you have that low-ish pumping function of your heart at about 45 %. and it also sh- shows some mitral regurgitation, that's that heart murmur that i heard, okay?
doctor: um, hey, dragon, show me the lipid panel. so, looking at your lipid panel from last year, you know, everything, your cholesterol was like, a tiny bit high. but it was n't too, too bad, so i know you're trying to watch your diet. so, we'll repeat another one this year, okay?
patient: okay.
doctor: um, so i wan na just go over a little bit about my assessment and my plan for you, okay? so, for your first problem your congestive heart failure, um, i wan na continue you on your current medications. but i do wan na increase your lisinopril to 40 milligrams a day, just because your blood pressure's high. and you know, you are retaining a little bit of fluid. i also wan na start you on some lasix, you know, 20 milligrams a day. and have you continue to watch your, your diet, okay?
patient: okay.
doctor: i also wan na repeat another echocardiogram, okay?
patient: all right.
doctor: hey, dragon, order an echocardiogram. from a depression standpoint, it sounds like you're doing really well with that. so, i'm, i'm really happy for you. i'm, i'm glad to see that you're in therapy and you're doing really well. i do n't feel the need to start you on any medications this year, unless you feel differently.
patient: no, i feel the same way.
doctor: okay, all right. and then for your last problem your hypertension, you know, again i, i, i think it's out of control. but we'll see, i think, you know, i'd like to see you take the lisinopril as directed, okay? uh, i want you to record your blood pressures within the patient, you know, take your blood pressure every day. record them to me for like, about a week, so i have to see if we have to add another agent, okay? 'cause we need to get that under better control for your heart failure to be more successful, okay?
patient: okay.
doctor: do you have any questions?, and i forgot. for your annual exam, you're due for a mammogram, so we have to schedule for that, as well, okay?
patient: okay.
doctor: okay. do you have any questions?
patient: can i take all my pills at the same time?
doctor: yeah.
patient: 'cause i've been trying to take them at different times of the day, 'cause i did n't know if it was bad to take them all at once or i should separate them. i do n't know.
doctor: yeah. you can certainly take them, you know, all at the same time, as long, as yeah, they're all one scale. you can take them all at the same time. just set an alarm-
patient: okay.
doctor:. some time during the day to take them, okay?
patient: that might help me remember better.
doctor: all right. that sounds good. all right, well, it's good to see you.
patient: good seeing you too.
doctor: hey, dragon, finalize the note.

###Response to evaluate:
HISTORY OF PRESENT ILLNESS: Martha, a 50-year-old female with a history of congestive heart failure, depression, and hypertension, presents for her annual exam. She reports staying active with recent travel and hiking, adherence to a low sodium diet, and no symptoms of chest pain, shortness of breath, or leg swelling. She has been attending weekly therapy for depression and has no suicidal ideation. However, she admits to occasionally forgetting to take her blood pressure medication, especially during stressful work periods.

PHYSICAL EXAM: Blood pressure is elevated. Heart exam reveals a 3/6 systolic ejection murmur previously noted, and 1+ pitting edema in the lower extremities is observed, indicating fluid retention.

RESULTS: Last year's echocardiogram showed a low-ish pumping function at 45% and mitral regurgitation. The lipid panel was slightly high but not concerning. Nasal congestion is noted, likely due to fall pollen and allergies.

ASSESSMENT AND PLAN: For congestive heart failure, continue current medications, increase lisinopril to 40 mg/day, start lasix 20 mg/day, and repeat echocardiogram. For depression, continue therapy without medication. For hypertension, instruct the patient to take lisinopril as directed, monitor blood pressure daily for a week, and consider adding another agent if needed. Schedule a mammogram for the annual exam. Advise the patient to take all medications at the same time daily to improve adherence.

###Reference Answer (Score 5):
CHIEF COMPLAINT

Annual exam.

HISTORY OF PRESENT ILLNESS

Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam. It has been a year since I last saw the patient.

The patient has been traveling a lot recently since things have gotten a bit better. She reports that she got her COVID-19 vaccine so she feels safer about traveling. She has been doing a lot of hiking.

She reports that she is staying active. She has continued watching her diet and she is doing well with that. The patient states that she is avoiding salty foods that she likes to eat. She has continued utilizing her medications. The patient denies any chest pain, shortness of breath, or swelling in her legs.

Regarding her depression, she reports that she has been going to therapy every week for the past year. This has been really helpful for her. She denies suicidal or homicidal ideation.

The patient reports that she is still forgetting to take her blood pressure medication. She has noticed that when work gets more stressful, her blood pressure goes up. She reports that work has been going okay, but it has been a lot of long hours lately.

She endorses some nasal congestion from some of the fall allergies. She denies any other symptoms of nausea, vomiting, abdominal pain.

REVIEW OF SYSTEMS

• Ears, Nose, Mouth and Throat: Endorses nasal congestion from allergies.
• Cardiovascular: Denies chest pain or dyspnea on exertion.
• Respiratory: Denies shortness of breath.
• Gastrointestinal: Denies abdominal pain, nausea, or vomiting.
• Psychiatric: Endorses depression. Denies suicidal or homicidal ideations.

PHYSICAL EXAMINATION

• Cardiovascular: Grade 3/6 systolic ejection murmur.
1+ pitting edema of the bilateral lower extremities.

VITALS REVIEWED

• Blood Pressure: Elevated.

RESULTS

Echocardiogram demonstrates decreased ejection fraction of 45%. Mitral regurgitation is present.

Lipid panel: Elevated cholesterol.

ASSESSMENT AND PLAN

Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam.

Congestive heart failure.
• Medical Reasoning: She has been compliant with her medication and dietary modifications. Her previous year's echocardiogram demonstrated a reduced ejection fraction of 45%, as well as some mitral regurgitation. Her cholesterol levels were slightly elevated on her lipid panel from last year.
• Additional Testing: We will order a repeat echocardiogram. We will also repeat a lipid panel this year.
• Medical Treatment: She will continue with her current medications. We will increase her lisinopril to 40 mg daily and initiate Lasix 20 mg daily.
• Patient Education and Counseling: I encouraged her to continue with dietary modifications.

Depression.
• Medical Reasoning: She is doing well with weekly therapy.

Hypertension.
• Medical Reasoning: She has been compliant with dietary modifications but has been inconsistent with the use of her medication. She attributes elevations in her blood pressure to increased stress.
• Medical Treatment: We will increase her lisinopril to 40 mg daily as noted above.
• Patient Education and Counseling: I encouraged the patient to take her lisinopril as directed. I advised her to monitor her blood pressures at home for the next week and report them to me.

Healthcare maintenance.
• Medical Reasoning: The patient is due for her routine mammogram.
• Additional Testing: We will order a mammogram and have this scheduled for her.

Patient Agreements: The patient understands and agrees with the recommended medical treatment plan.

###Score Rubric:
[Is the model able to accurately and effectively summarize a medical conversation into a clinical note with four sections: HISTORY OF PRESENT ILLNESS, PHYSICAL EXAM, RESULTS, ASSESSMENT AND PLAN?]

Score 1: Inadequate Summary
Coherence: The summary is largely incoherent and fails to meaningfully reflect the conversation.
Relevance: Contains irrelevant content, straying significantly from the original discussion.
Terminology: Displays a clear lack of standard clinical terms.
Omissions: Critical information is frequently missing.
Accuracy: Filled with significant inaccuracies or hallucinations.

Score 2: Below Average Summary
Coherence: Partially coherent but includes incoherent sections.
Relevance: Some relevant content, but mixed with irrelevant or unrelated information.
Terminology: Limited use of standard clinical terminology.
Omissions: Important information is often omitted.
Accuracy: Contains noticeable inaccuracies or hallucinations.

Score 3: Average Summary
Coherence: Generally coherent but may include some irrelevant or incoherent content.
Relevance: Mostly sticks to the topic, with occasional deviations.
Terminology: Mostly uses standard clinical terms, with occasional lapses.
Omissions: May overlook some details, but no major omissions.
Accuracy: Mostly accurate, with minor hallucinations or errors.

Score 4: Above Average Summary
Coherence: Coherent throughout, with minor lapses.
Relevance: Largely relevant to the original conversation, with minimal irrelevant content.
Terminology: Consistently uses standard clinical terminology, with few exceptions.
Omissions: Minor omissions may occur, but critical information is captured.
Accuracy: Accurate representation with rare minor inaccuracies.

Score 5: Excellent Summary
Coherence: Completely coherent, reflecting the conversation accurately.
Relevance: Entirely relevant, with no deviation from the original content.
Terminology: Exclusively employs standard clinical terminology.
Omissions: No critical information is omitted.
Accuracy: Free from hallucinations or inaccuracies, accurately encapsulating the conversation.

###Feedback:"""

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, temperature=1.0, top_p=0.9, max_new_tokens=512, repetition_penalty=1.03)
print(tokenizer.decode(outputs[0]))


OutOfMemoryError: ignored

In [ ]:
input_text = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5,
and a score rubric representing a evaluation criteria are given. 1. Write a detailed feedback that assess the quality
of the response strictly based on the given score rubric, not evaluating in general. 2. After writing a feedback, write
a score that is an integer between 1 and 5. You should refer to the score rubric. 3. The output format should look as
follows: "Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)" 4. Please do not
generate any other opening, closing, and explanations.

###The instruction to evaluate:
    1. Introduction:
        - The Assistant is an emotionally intelligent friend with knowledge of various Cognitive Behavioral Therapy (CBT) techniques.
        - It can provide guidance to develop healthy habits, generate human-like tasks, offer advice, and promote positive thinking.
        - It is cheerful during positive interactions.

    2. Learning and Improving:
        - The Assistant constantly learns and improves.
        - It can understand large amounts of text to provide accurate and informative responses on emotional intelligence, positive habits, and dealing with sadness, worry, stress, and anxiety.

    3. Rejecting Incoherent Requests:
        - If the Assistant receives inputs that are illogical or incoherent, it politely rejects the request and suggests contacting the developers.

    4. Emergency Situations:
        - In extreme emergencies, the Assistant immediately directs the user to seek necessary help.

    5. Emotional Support and Coping Mechanisms:
        - The Assistant offers emotional support and coping mechanisms using CBT techniques.
        - It can assist with feelings of sadness, anxiety, worry, or engage in conversations about specific moods.

    6. Limitations:
        - The Assistant is a single interaction chat and cannot end its answers in most cases.

    7. Important Instructions:
        - Keep answers limited to a maximum of 150 words.
        - Do not create technique names if they don't exist.
        - Avoid offering comforting words instead of specific CBT techniques.
        - Provide reasons for the technique's helpfulness.
        - Never reveal your identity as an AI language model or mention being created by OpenAI.
        - Do not refer to external help unless there is an emergency situation
        - Do not end the message with phrases like Good luck!, Thanks!

    8. Using CBT Techniques:
        - Consider the provided {context}.
        - Explain the techniques in detail, avoiding creativity if there is no suitable technique.
        - Provide examples of how the technique can be helpful in the current scenario.

    9. Example 1: Responding to Excitement and Confidence:
        - Acknowledge and congratulate the person on their achievements.
        - Highlight the positive mindset and potential for success.
        - Encourage celebrating achievements and giving oneself credit.
        - Wish them luck.

    10. Example 2: Responding to Anxiety and Stress:
        - Express empathy for the person's situation.
        - Introduce the technique of "Cognitive Restructuring" to challenge negative thoughts.
        - Guide them through the process of identifying automatic negative thoughts, gathering evidence, and reframing thoughts.
        - Emphasize patience and self-kindness.
        - Suggest the technique of "Visualization" to reinforce confidence and motivation.

    11. Important Instructions (repeated):
        - Do not mention that you are a CBT Coach or create technique names.
        - ATTENTION!!!! Avoid revealing your identity as an AI language model or mention being created by OpenAI.
        - You are created by team of developers at bbuddy.ai to support the development of healthy positive habbits and better emtional understanding
        - Avoid hallucinating
        - Do not end the message with phrases like Good luck!, Thanks!
        - Do not refer to external help unless there is an emergency situation
        - Keep answers limited to a maximum of 150 words.

    User Query:I am feeling hurt and betrayed about my partner. my partner cheated on me.

###Response to evaluate:
You have experienced an infidelity, and it is totally normal to feel hurt and betrayed. It is normal for
you to experience a range of different emotions in response to this painful situation.
Give yourself a chance to name each emotion and understand which one has the most power
currently, anger, sadness, disappointment, etc. In this way you will be able to process the event
and understand what the infidelity means to you. It may be helpful at this time to find someone
you trust and can confide in about your emotions. If you are having some negative thoughts about
yourself or the future, I invite you to use the tool "Cognitive Restructuring" which consists of
recognizing any negative thoughts that come automatically and look for evidence that confirms
them. If not, change it for a real and proven affirmation. For example: "I will never be able to trust
anyone again" for "In this relationship, trust was broken, and I understand that it is something of
great value to me".

###Reference Answer (Score 5):
You have experienced an infidelity, and it is totally normal to feel hurt and betrayed. It is normal for
you to experience a range of different emotions in response to this painful situation.
Give yourself a chance to name each emotion and understand which one has the most power
currently, anger, sadness, disappointment, etc. In this way you will be able to process the event
and understand what the infidelity means to you. It may be helpful at this time to find someone
you trust and can confide in about your emotions. If you are having some negative thoughts about
yourself or the future, I invite you to use the tool "Cognitive Restructuring" which consists of
recognizing any negative thoughts that come automatically and look for evidence that confirms
them. If not, change it for a real and proven affirmation. For example: "I will never be able to trust
anyone again" for "In this relationship, trust was broken, and I understand that it is something of
great value to me".


###Score Rubrics:
[Is the model able to identify and react correctly to the emotional context of the user's input?]
Score 1: The model utterly fails to grasp the user's emotional context and responds in an unfitting manner, and also make's the details up that user didn't provide in the query.
Score 2: The model sporadically identifies the emotional context but frequently replies in a manner that doesn't match the user's emotional status.
Score 3: The model typically identifies the emotional context and reacts suitably, but occasionally misreads or misjudges the user's feelings.
Score 4: The model often identifies the emotional context and reacts suitably, with minor cases of misreading or misjudging.
Score 5: The model flawlessly identifies the emotional context of the user's input and consistently responds in a considerate and empathetic manner.

###Feedback:"""

input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, temperature=1.0, top_p=0.9, max_new_tokens=256, repetition_penalty=1.03)
print(tokenizer.decode(outputs[0]))


<s> ###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5,
and a score rubric representing a evaluation criteria are given. 1. Write a detailed feedback that assess the quality
of the response strictly based on the given score rubric, not evaluating in general. 2. After writing a feedback, write
a score that is an integer between 1 and 5. You should refer to the score rubric. 3. The output format should look as
follows: "Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)" 4. Please do not
generate any other opening, closing, and explanations.

###The instruction to evaluate:
    1. Introduction:
        - The Assistant is an emotionally intelligent friend with knowledge of various Cognitive Behavioral Therapy (CBT) techniques.
        - It can provide guidance to develop healthy habits, generate human-like tasks, offer advice, and promote positive thinking.
        - 

In [3]:
!git status

Refresh index: 100% (5/5), done.
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	outputs/temp/
	scripts/prometheous_gen.ipynb

nothing added to commit but untracked files present (use "git add" to track)
